# Pegando a lista de servidores

In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from threading import Thread
base = 'csv/'

In [25]:
def requestHTMLPage(url):
    r = requests.get(url)
    while r.status_code != 200:
        time.sleep(10)
        r = requests.get(url)
        print("Loop em potencial")
    return r.content

In [5]:
# para pegar os servidores da UFRN
urlBaseSite = "http://www.portaldatransparencia.gov.br/"
urlBaseOrgaoExercicio = "servidores/OrgaoExercicio-ListaServidores.asp?CodOrg=26243&Pagina="
currentMonth = 4

In [30]:
content = requestHTMLPage(urlBaseSite+urlBaseOrgaoExercicio)
soup = BeautifulSoup(content, "html.parser")
pages = soup.find_all("div", id = "paginacao")

In [33]:
# get the greater page number of functionaries
for p in pages:
    maxPage = p.select('a')[1].get('onclick')
maxPage = int(maxPage[maxPage.find("(")+1:maxPage.find(")")])

In [34]:
def getName(sumarry):
    sumarry = sumarry.find_all('td')
    iterDetails = iter(sumarry)
    while True:
        try:
            currentDetail = next(iterDetails).getText()
            if 'Nome:' in currentDetail:
                name = next(iterDetails).getText().strip()
        except StopIteration:
            break
    return name

def getMoneyValue(soup):
    basicRemuneration = soup.find_all('td')
    iterDetails = iter(basicRemuneration)
    money = ""
    while True:
        try:
            currentDetail = next(iterDetails).getText()
            if 'Remuneração básica bruta' in currentDetail:
                money = next(iterDetails).getText().strip()
        except StopIteration:
            break
    return money

def getMoneyInfo(moneyUrl):
    money = "-1000"
    for i in range(1, currentMonth+1):
        moneyPage = requestHTMLPage(urlBaseSite + moneyUrl + "&Ano=2017&Mes=" + str(i))
        if 'Servidor sem ficha' in str(moneyPage):
            continue
        else:
            soup = BeautifulSoup(moneyPage, "html.parser")
            money = getMoneyValue(soup)
            break
    return money

def getSumarryInfo(soup):
    sumarry = soup.find_all('div', id='resumo')[0]
    moneyUrl = sumarry.find_all('a')[0].get('href')
    name = getName(sumarry)
    money = getMoneyInfo(moneyUrl)
    return name, money

def getGeneralInfo(soup):
    details = soup.find_all('td')
    iterDetails = iter(details)
    post = ""
    class_ = ""
    level = ""
    org = ""
    while True:
        try:
            currentDetail = next(iterDetails).getText()
            if 'Cargo Emprego:' in currentDetail:
                post = next(iterDetails).getText().strip()
            elif 'Classe:' in currentDetail:
                class_ = next(iterDetails).getText().strip()
            elif 'Nível:' in currentDetail:
                level = next(iterDetails).getText().strip()
            elif 'UORG:' in currentDetail:
                org = next(iterDetails).getText().strip()
        except StopIteration:
            break
    return post, class_, level, org

def getFunctionaryInfo(functionaryUrl, infos):
    info = requestHTMLPage(urlBaseSite + 'servidores/' + functionaryUrl)
    soup = BeautifulSoup(info, "html.parser")
    name, money = getSumarryInfo(soup)
    post, class_, level, org = getGeneralInfo(soup)
    infos.append((name, money, post, class_, level, org))

In [35]:
def getFuncionaryUrls(pageNumber):
    content = requestHTMLPage(urlBaseSite + urlBaseOrgaoExercicio + str(pageNumber))
    soup = BeautifulSoup(content, "html.parser")
    funcionaries = soup.find_all('div', id='listagem')
    funcionariesList = []
    for l in funcionaries:
        table = l.find_all('tr')
        table = table[1:]
        for funcionary in table:
            funcionariesList.append(funcionary.find_all('a')[0].get('href'))
    return funcionariesList

In [36]:
def toString(name, money, post, class_, level, org):
    return name + ';' + money + ';' + post + ';' + class_ + ';' + level + ';' + org + '\n'

def saveCSV(infos):
    file = open(base + 'servers.csv', 'w')
    file.write("name;money;post;class;level;org\n")
    for i in infos:
        file.write(toString(*i))
    file.close()

In [38]:
def callGetFunctionaryInfo(urls, infos):
    threads = list()
    for url in urls:
        thread = Thread(target=getFunctionaryInfo, args=(url, infos))
        threads.append(thread)
    for t in threads:
        t.start()
    for t in threads:
        t.join()

maxLen = 30
infos = []
urls = []
for i in range(0, maxPage): # maxPage
    urls += getFuncionaryUrls(i + 1)
    if len(urls) >= maxLen:
        callGetFunctionaryInfo(urls, infos)
        urls = []
        print("Current page " + str(i + 1))
if len(urls) > 0:
    callGetFunctionaryInfo(urls, infos)
print("Finished")
saveCSV(infos)

Current page 2
Current page 4
Current page 6
Current page 8
Current page 10
Current page 12
Current page 14
Current page 16
Current page 18
Current page 20
Current page 22
Current page 24
Current page 26
Current page 28
Current page 30
Current page 32
Current page 34
Current page 36
Current page 38
Current page 40
Current page 42
Current page 44
Current page 46
Current page 48
Current page 50
Current page 52
Current page 54
Current page 56
Current page 58
Current page 60
Current page 62
Current page 64
Current page 66
Current page 68
Current page 70
Current page 72
Current page 74
Current page 76
Current page 78
Current page 80
Current page 82
Current page 84
Current page 86
Current page 88
Current page 90
Current page 92
Current page 94
Current page 96
Current page 98
Current page 100
Current page 102
Current page 104
Current page 106
Current page 108
Current page 110
Current page 112
Current page 114
Current page 116
Current page 118
Current page 120
Current page 122
Current page 124

Exception in thread Thread-9835:
Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 298, in _error_catcher
    yield
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 380, in read
    data = self._fp.read(amt)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 449, in read
    n = self.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 493, in readinto
    n = self.fp.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/models.py", line 703, in generate
    for chunk in self.raw.stream(chunk_size, decode

Current page 334
Current page 336
Current page 338
Current page 340
Current page 342
Current page 344
Current page 346
Current page 348
Current page 350
Current page 352
Current page 354
Current page 356
Current page 358
Current page 360
Current page 362
Current page 364
Current page 366
Current page 368
Current page 370
Current page 372


Exception in thread Thread-10419:
Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 298, in _error_catcher
    yield
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 380, in read
    data = self._fp.read(amt)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 449, in read
    n = self.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 493, in readinto
    n = self.fp.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/models.py", line 703, in generate
    for chunk in self.raw.stream(chunk_size, decod

Current page 374


Exception in thread Thread-10445:
Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 298, in _error_catcher
    yield
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/response.py", line 380, in read
    data = self._fp.read(amt)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 449, in read
    n = self.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/http/client.py", line 493, in readinto
    n = self.fp.readinto(b)
  File "/home/danilo/anaconda3/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/danilo/anaconda3/lib/python3.6/site-packages/requests/models.py", line 703, in generate
    for chunk in self.raw.stream(chunk_size, decod

Current page 376
Current page 378
Current page 380
Current page 382
Current page 384
Current page 386
Current page 388
Current page 390
Current page 392
Current page 394
Current page 396
Current page 398
Current page 400
Current page 402
Current page 404
Current page 406
Current page 408
Current page 410
Current page 412
Finished
